In [337]:
from gurobipy import *
import numpy as np

In [338]:
class RSP_conj:
    
    def __init__(self):
        return
    
    def prog(self):
        
        options = ['R', 'S', 'P']
        m = Model("RSP-Program")
        # we will maximize the slack vars in the strict inequalities
        m.setAttr("ModelSense", -1)
        # allow non-convex quadratic constraints
        m.setParam("NonConvex", 2)
        
        ## decision variables
        
        # the player's distribution
        p = m.addVars(options, vtype = GRB.CONTINUOUS, name = 'p')
        
        # the opponent distribution
        p_op = m.addVars(options, vtype = GRB.CONTINUOUS, name = 'p*')
        
        # vars for the strict inequalitys
        epsilon = m.addVars(['c','b'], vtype = GRB.CONTINUOUS, name = 'epsilon', obj = [10.0,1.0])
        
        # the strategy where the player picks an option randomly
        random_strategy = {o: 1/3 for o in options}

        ## constraints
        
        # prob dist constraints 
        m.addConstr(quicksum(p[i] for i in options) == 1)
        m.addConstr(p_op['S'] == 0)        
        m.addConstr(quicksum(p_op[i] for i in options) == 1)
        
        # aux vars to handle abs value in the tv constr
        d_p = m.addVars(options, vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'd_p')
        d_op = m.addVars(options, vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'd_op')
        a_p = m.addVars(options, vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'a_p')
        a_op = m.addVars(options, vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'a_op')
        # the total variation distance between the player's strategy and the opponent's must be less the random strategy and the opponent's
        m.addConstrs(d_p[i] == p[i] - p_op[i] for i in options)
        m.addConstrs(a_p[i] == abs_(d_p[i]) for i in options)
        m.addConstrs(d_op[i] == random_strategy[i] - p_op[i] for i in options)
        m.addConstrs(a_op[i] == abs_(d_op[i]) for i in options)
        m.addConstr(quicksum(a_p[i] for i in options) + 0.01  <= quicksum(a_op[i] for i in options))
        
        # l2 norm constraint
        m.addConstr(quicksum((p[i] - p_op[i])*(p[i] - p_op[i]) for i in options) + 0.01 <= quicksum((1/3 - p_op[i])*(1/3 - p_op[i]) for i in options))
        
        # the utility for the cyclic response must be negative
        x = m.addVars(options, vtype = GRB.CONTINUOUS, lb = - GRB.INFINITY, name = 'x')
        m.addConstr(x['R'] == p_op['R']-p_op['S'])
        m.addConstr(x['S'] == p_op['S']-p_op['P'])
        m.addConstr(x['P'] == p_op['P']-p_op['R'])
        m.addQConstr(p['R']*x['R'] + p['S']*x['S'] + p['P']*x['P']  + epsilon['c'] <= 0)
        
        # the utility for best response must be negative 
        indic = m.addVars(options, vtype = GRB.BINARY, name = 'i')
        
        z = m.addVar(vtype = GRB.CONTINUOUS, name = 'z')
        m.addConstr(z >= p['S'] - p['P']) 
        m.addConstr(z >= p['P'] - p['R'])
        m.addConstr(z >= p['R'] - p['S'])
        m.addConstr((indic['R'] == 1) >> (z == p['S'] - p['P']))
        m.addConstr((indic['S'] == 1) >> (z == p['P'] - p['R']))
        m.addConstr((indic['P'] == 1) >> (z == p['R'] - p['S']))
        
        m.addConstr(quicksum(indic[i] for i in options) == 1)
        
        m.addQConstr(indic['R']*x['S'] + indic['S']*x['P'] + indic['P']*x['R']  + epsilon['b'] <= 0)
        
        ## solve
        m.optimize()
        
        m.setParam("SolutionNumber", 2)
                   
        self.solution = m
        return

In [339]:
example = RSP_conj()
example.prog()

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 17 rows, 27 columns and 49 nonzeros
Model fingerprint: 0x088e377c
Model has 3 quadratic constraints
Model has 9 general constraints
Variable types: 24 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [7e-01, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-02, 1e+00]
  QRHS range       [3e-01, 3e-01]
Presolve added 9 rows and 4 columns
Presolve time: 0.00s
Presolved: 42 rows, 38 columns, 111 nonzeros
Presolved model has 3 SOS constraint(s)
Presolved model has 1 quadratic constraint(s)
Presolved model has 4 bilinear constraint(s)
Variable types: 27 continuous, 11 integer (11 binary)

Root relaxation: objective 1.100000e+01, 29 iterations, 0.00 seconds

    Nodes    |    Current Node   

In [340]:
print('Oracle Distribution - Rock: %g, Scissors: %g, Papper: %g' % 
      tuple([example.solution.getVarByName('p[%s]' % i).X for i in ['R','S','P']]))

print('Opponent True Distribution - Rock: %g, Scissors: %g, Papper: %g' % 
      tuple([example.solution.getVarByName('p*[%s]' % i).X for i in ['R','S','P']]))


utility = example.solution.getVarByName('i[R]').X*(example.solution.getVarByName('p*[S]').X- example.solution.getVarByName('p*[P]').X) \
+ example.solution.getVarByName('i[S]').X*(example.solution.getVarByName('p*[P]').X- example.solution.getVarByName('p*[R]').X) \
+ example.solution.getVarByName('i[P]').X*(example.solution.getVarByName('p*[R]').X- example.solution.getVarByName('p*[S]').X)

print('Best response utility: ' + str(utility))

utility = example.solution.getVarByName('p[R]').X*(example.solution.getVarByName('p*[R]').X- example.solution.getVarByName('p*[S]').X) \
+ example.solution.getVarByName('p[S]').X*(example.solution.getVarByName('p*[S]').X- example.solution.getVarByName('p*[P]').X) \
+ example.solution.getVarByName('p[P]').X*(example.solution.getVarByName('p*[P]').X- example.solution.getVarByName('p*[R]').X)

print('Cyclic response utility: ' + str(utility))

Oracle Distribution - Rock: 0.276785, Scissors: 0.256537, Papper: 0.466677
Opponent True Distribution - Rock: 0.605119, Scissors: 0, Papper: 0.394881
Best response utility: -0.21023753692945468
Cyclic response utility: -0.031926803198328316


In [341]:
result1 = abs(example.solution.getVarByName('p[R]').X- example.solution.getVarByName('p*[R]').X) \
+ abs(example.solution.getVarByName('p[S]').X- example.solution.getVarByName('p*[S]').X) \
+ abs(example.solution.getVarByName('p[P]').X- example.solution.getVarByName('p*[P]').X) 
print('p - p* : tv ' + str(result1))
result2 = abs(1/3 - example.solution.getVarByName('p*[R]').X) \
+ abs(1/3 - example.solution.getVarByName('p*[S]').X) \
+ abs(1/3 - example.solution.getVarByName('p*[P]').X) 
print('1/3 - p* : tv ' + str(result2))
print(result2 - result1)

p - p* : tv 0.6566666666666666
1/3 - p* : tv 0.6666666666666667
0.01000000000000012


In [342]:
result1 = (example.solution.getVarByName('p[R]').X- example.solution.getVarByName('p*[R]').X)**2 \
+ (example.solution.getVarByName('p[S]').X- example.solution.getVarByName('p*[S]').X)**2 \
+ (example.solution.getVarByName('p[P]').X- example.solution.getVarByName('p*[P]').X)**2 
print('p - p* : l2 ' + str(result1))
result2 = (1/3 - example.solution.getVarByName('p*[R]').X)**2 \
+ (1/3 - example.solution.getVarByName('p*[S]').X)**2 \
+ (1/3 - example.solution.getVarByName('p*[P]').X)**2 
print('1/3 - p* : l2 ' + str(result2))
print(result2 - result1)

p - p* : l2 0.17876887683052128
1/3 - p* : l2 0.1887665776337486
0.009997700803227316


In [343]:
for v in example.solution.getVars():
    print('%s = %g' % (v.varName, v.x))

p[R] = 0.276785
p[S] = 0.256537
p[P] = 0.466677
p*[R] = 0.605119
p*[S] = 0
p*[P] = 0.394881
epsilon[c] = 0.0319277
epsilon[b] = 0.210238
d_p[R] = -0.328333
d_p[S] = 0.256537
d_p[P] = 0.0717959
d_op[R] = -0.271785
d_op[S] = 0.333333
d_op[P] = -0.0615479
a_p[R] = 0.328333
a_p[S] = 0.256537
a_p[P] = 0.0717959
a_op[R] = 0.271785
a_op[S] = 0.333333
a_op[P] = 0.0615479
x[R] = 0.605119
x[S] = -0.394881
x[P] = -0.210238
i[R] = -0
i[S] = 1
i[P] = -0
z = 0.189892


In [344]:
# suboptimal solutions
print('Oracle Distribution - Rock: %g, Scissors: %g, Papper: %g' % 
      tuple([example.solution.getVarByName('p[%s]' % i).Xn for i in ['R','S','P']]))
print('Opponent True Distribution - Rock: %g, Scissors: %g, Papper: %g' % 
      tuple([example.solution.getVarByName('p*[%s]' % i).Xn for i in ['R','S','P']]))
utility = example.solution.getVarByName('i[R]').Xn*(example.solution.getVarByName('p*[S]').Xn- example.solution.getVarByName('p*[P]').Xn) \
+ example.solution.getVarByName('i[S]').Xn*(example.solution.getVarByName('p*[P]').Xn- example.solution.getVarByName('p*[R]').Xn) \
+ example.solution.getVarByName('i[P]').Xn*(example.solution.getVarByName('p*[R]').Xn- example.solution.getVarByName('p*[S]').Xn)
print('Best response utility: ' + str(utility))
utility = example.solution.getVarByName('p[R]').Xn*(example.solution.getVarByName('p*[R]').Xn- example.solution.getVarByName('p*[S]').Xn) \
+ example.solution.getVarByName('p[S]').Xn*(example.solution.getVarByName('p*[S]').Xn- example.solution.getVarByName('p*[P]').Xn) \
+ example.solution.getVarByName('p[P]').Xn*(example.solution.getVarByName('p*[P]').Xn- example.solution.getVarByName('p*[R]').Xn)
print('Cyclic response utility: ' + str(utility))
result1 = (example.solution.getVarByName('p[R]').Xn- example.solution.getVarByName('p*[R]').Xn)**2 \
+ (example.solution.getVarByName('p[S]').Xn- example.solution.getVarByName('p*[S]').Xn)**2 \
+ (example.solution.getVarByName('p[P]').Xn- example.solution.getVarByName('p*[P]').Xn)**2 
print('p - p* : l2 norm ' + str(result1))
result2 = (1/3 - example.solution.getVarByName('p*[R]').Xn)**2 \
+ (1/3 - example.solution.getVarByName('p*[S]').Xn)**2 \
+ (1/3 - example.solution.getVarByName('p*[P]').Xn)**2 
print('1/3 - p* : l2 norm ' + str(result2))
print(result2 - result1)

Oracle Distribution - Rock: 0.276036, Scissors: 0.255683, Papper: 0.468281
Opponent True Distribution - Rock: 0.604369, Scissors: 0, Papper: 0.395631
Best response utility: -0.20873862228793444
Cyclic response utility: -0.032076753239038455
p - p* : l2 norm 0.1784547434589634
1/3 - p* : l2 norm 0.18845257288399914
0.009997829425035742
